In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import random
import time
import tiktoken
from datetime import datetime


# .env 파일 로드
load_dotenv()

True

# 데이터 가져오기

In [3]:

investing_path = r'C:\STFOPorject\STFO-2\Data\Investing_Data.json'
hankyung_path = r'C:\STFOPorject\STFO-2\Data\Hankyung_Data.json'

In [29]:
# 현재 날짜와 시간 가져오기
current_time = datetime.now()

# 현재 날짜와 시간을 "년-월-일-시-분-초" 형식으로 출력
formatted_time = current_time.strftime("%Y-%m-%d-%H:%M:%S")

print("현재 날짜와 시간:", formatted_time)

현재 날짜와 시간: 2024-12-10-11:04:08


In [4]:
df1 = pd.read_json(investing_path, orient='records', encoding='utf-8', dtype='string')
df1.head()

,news_title,news_first_upload_time,news_last_upload_time,author,news_content,news_url,news_website,note
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,<NA>,<NA>,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,<NA>,<NA>,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,<NA>,<NA>,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,<NA>,<NA>,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,<NA>,<NA>,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>


In [5]:
df2 = pd.read_json(hankyung_path, orient='records', encoding='utf-8', dtype='string')
df2.head()

,news_title,news_first_upload_time,news_last_upload_time,author,news_content,news_url,news_website,note
0,"해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐",2024-12-07 PM 07:06,2024-12-07 PM 07:06,양한나 기자,인도 및 신흥시장 전문 웹3 벤처캐피털 해시드 이머전트(Hashed Emergent...,https://www.hankyung.com/article/202412071413g,Hankyung,<NA>
1,비트코인 10만달러 재탈환…이더리움도 '껑충',2024-12-07 AM 11:13,2024-12-07 AM 11:13,<NA>,가상화폐 대장주 비트코인 가격이 6일(현지시간) 10만 달러선을 재탈환했다. 시가총...,https://www.hankyung.com/article/2024120709145,Hankyung,<NA>
2,비트코인 10만달러 재돌파…'이더리움의 시간' 오나,2024-12-07 AM 09:54,2024-12-07 AM 09:54,고정삼 기자,암호화폐 대장주 비트코인이 6일(현지시간) 10만달러선 재탈환에 성공했다. 시가총액...,https://www.hankyung.com/article/2024120707037,Hankyung,<NA>
3,"""비트코인 전략비축? 헛소리""…美 전 재무장관 '일침'",2024-12-06 AM 10:15,2024-12-06 AM 10:15,<NA>,로렌스 서머스 전 미국 재무부 장관이 가상화폐 비트코인을 원유처럼 전략비축하자는 주...,https://www.hankyung.com/article/2024120685065,Hankyung,<NA>
4,"비트코인, 다시 10만달러 아래로…트럼프 '환호'",2024-12-06 AM 06:35,2024-12-06 AM 06:58,<NA>,가상화폐 비트코인이 지난 4일(현지시간) 사상 처음 10만 달러를 돌파했지만 하루만...,https://www.hankyung.com/article/2024120679115,Hankyung,<NA>


In [6]:
# 행 단위로 결합
df_combined = pd.concat([df1, df2], ignore_index=True)

df_combined.head()

,news_title,news_first_upload_time,news_last_upload_time,author,news_content,news_url,news_website,note
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,<NA>,<NA>,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,<NA>,<NA>,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,<NA>,<NA>,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,<NA>,<NA>,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,<NA>,<NA>,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing,<NA>


In [7]:
# 필요한 열만 선택하여 새로운 데이터프레임 생성
df_combined = df_combined[['news_title', 'news_first_upload_time', 'news_content', 'news_url', 'news_website']]

# 결과 확인
df_combined.head()

,news_title,news_first_upload_time,news_content,news_url,news_website
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing


In [8]:
# 내림차순 정렬
df_sorted = df_combined.sort_values(by="news_first_upload_time", ascending=False)
df_sorted.head(10)

,news_title,news_first_upload_time,news_content,news_url,news_website
0,XRP 가격이 10% 상승,2024-12-08 AM 01:06,Investing.com - XRP 가격이 Investing.com Index에서 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
1,"스탠다드차타드 ""비트코인, 내년까지 20만달러 돌파 가능""",2024-12-07 PM 08:45,비트코인 (BTC)이 내년까지 20만달러를 돌파할 수 있다는 전망이 나왔다. 6일(...,https://kr.investing.com/news/cryptocurrency-n...,Investing
69916,"해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐",2024-12-07 PM 07:06,인도 및 신흥시장 전문 웹3 벤처캐피털 해시드 이머전트(Hashed Emergent...,https://www.hankyung.com/article/202412071413g,Hankyung
2,"""바이낸스 상장으로 9.5배 수익""…50대 노린 꼼수 사기 기승",2024-12-07 PM 06:00,최근 가상자산 가격이 폭등하면서 ‘포모(Fear of Missing Out·FOMO...,https://kr.investing.com/news/cryptocurrency-n...,Investing
69917,비트코인 10만달러 재탈환…이더리움도 '껑충',2024-12-07 AM 11:13,가상화폐 대장주 비트코인 가격이 6일(현지시간) 10만 달러선을 재탈환했다. 시가총...,https://www.hankyung.com/article/2024120709145,Hankyung
69918,비트코인 10만달러 재돌파…'이더리움의 시간' 오나,2024-12-07 AM 09:54,암호화폐 대장주 비트코인이 6일(현지시간) 10만달러선 재탈환에 성공했다. 시가총액...,https://www.hankyung.com/article/2024120707037,Hankyung
3,"[코인 리포트] ""프로젝트 고문이 '가상자산 차르'""…제로엑스(ZRX) 40% 급등",2024-12-07 AM 03:13,백악관 내 가상자산 정책 전담 직책으로 신설돼 업계의 이목이 집중됐던 ‘가상자산 차...,https://kr.investing.com/news/cryptocurrency-n...,Investing
4,"폴 앳킨스 SEC 위원장 지명 소식에 업계 ""대환영""",2024-12-07 AM 01:22,폴 앳킨스 전 미국 증권거래위원회(SEC) 위원의 차기 SEC 위원장 지명 소식에 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
5,"[점심브리핑] 파이낸셜타임즈 ""우리 기사 때문에 BTC 안 샀다면 죄송""",2024-12-06 PM 10:38,주요 가상자산이 혼조세다. 6일 오후 12시 56분 빗썸 기준 비트코인(BTC)은 ...,https://kr.investing.com/news/cryptocurrency-n...,Investing
8,"[디센터 인터뷰] 리플랩스 ""스테이블코인 'RLUSD' 출시 임박…RWA 시장 공략""",2024-12-06 PM 09:08,“도널드 트럼프 미 대통령 당선인의 승리는 실물연계자산(RWA) 시장 활성화의 분수...,https://kr.investing.com/news/cryptocurrency-n...,Investing


In [9]:
df_sorted.isnull().sum()


news_title                151
news_first_upload_time    151
news_content                0
news_url                    0
news_website                0
dtype: int64

In [10]:
# news_title 열에 NaN 값이 있는 행 삭제
df_sorted_cleaned = df_sorted.dropna(subset=["news_title"])

# 결과 확인
print(df_sorted_cleaned)

                                     news_title news_first_upload_time  \
0                                XRP 가격이 10% 상승    2024-12-08 AM 01:06   
1              스탠다드차타드 "비트코인, 내년까지 20만달러 돌파 가능"    2024-12-07 PM 08:45   
69916        해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐    2024-12-07 PM 07:06   
2           "바이낸스 상장으로 9.5배 수익"…50대 노린 꼼수 사기 기승    2024-12-07 PM 06:00   
69917                 비트코인 10만달러 재탈환…이더리움도 '껑충'    2024-12-07 AM 11:13   
...                                         ...                    ...   
69911          바이낸스 CEO "바이낸스, 마진 거래 서비스 출시 예정"    2019-05-08 PM 06:00   
69912           유명 암호화폐 트레이더 "비트파이넥스 LEO 토큰 주의"    2019-05-08 PM 05:57   
69913             日 거래소 비트플라이어, 사용자 자산 긴급 점검 진행    2019-05-08 PM 05:54   
69914  ETH 코어 개발자 "블록 리오그 여부, 바이낸스 아닌 커뮤니티의 결정"    2019-05-08 PM 05:33   
69915             페이팔 CFO "블록체인·암호화폐, 아직은 먼 미래"    2019-05-08 PM 05:19   

                                            news_content  \
0      Investing.com - XRP 가격이 Investing.com Index에

In [11]:
df_sorted_cleaned.isnull().sum()


news_title                0
news_first_upload_time    0
news_content              0
news_url                  0
news_website              0
dtype: int64

In [12]:
# 데이터프레임 생성
df_sorted_cleaned 

# 1. 날짜 문자열을 datetime 형식으로 변환
def parse_datetime(date_str):
    # AM/PM 형식을 포함한 날짜를 변환
    return datetime.strptime(date_str, "%Y-%m-%d %p %I:%M")

df_sorted_cleaned["news_first_upload_time"] = df_sorted_cleaned["news_first_upload_time"].apply(parse_datetime)

# 2. 11월 1일부터 최근 데이터까지 필터링
start_date = datetime(2024, 11, 1)  # 11월 1일
filtered_df = df_sorted_cleaned[df_sorted_cleaned["news_first_upload_time"] >= start_date]

# 3. 결과 확인
print(filtered_df)

                                          news_title news_first_upload_time  \
0                                     XRP 가격이 10% 상승    2024-12-08 01:06:00   
1                   스탠다드차타드 "비트코인, 내년까지 20만달러 돌파 가능"    2024-12-07 20:45:00   
69916             해시드 이머전트, '인도 블록체인 위크 2024' 성황리 마쳐    2024-12-07 19:06:00   
2                "바이낸스 상장으로 9.5배 수익"…50대 노린 꼼수 사기 기승    2024-12-07 18:00:00   
69917                      비트코인 10만달러 재탈환…이더리움도 '껑충'    2024-12-07 11:13:00   
...                                              ...                    ...   
70052  "도시락 먹고 이더리움 받자"…빗썸·이마트24, '이더리움 스테이킹 도시락' 출시    2024-11-01 11:24:00   
70053                  최고가 넘보던 비트코인 '휘청'...나스닥 하락 영향    2024-11-01 07:14:00   
382               비트코인 랠리 주춤… 월가 리스크 회피 심리 속 암호화폐 하락    2024-11-01 01:26:00   
384    이두희 모던라이언 대표 "티켓팅에 NFT 안 쓸 이유 없어…내년에 암표 없앨 것"    2024-11-01 01:24:00   
383          가상자산 시장, 하루 6조 원 오갔다…상반기 거래소 영업이익 2배 껑충    2024-11-01 01:18:00   

                                            news_co

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_24816\2872951450.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted_cleaned["news_first_upload_time"] = df_sorted_cleaned["news_first_upload_time"].apply(parse_datetime)


In [13]:
filtered_df["news_first_upload_time"].tail(10)

380     2024-11-01 19:00:00
379     2024-11-01 18:53:00
70050   2024-11-01 18:16:00
70051   2024-11-01 17:12:00
381     2024-11-01 15:35:00
70052   2024-11-01 11:24:00
70053   2024-11-01 07:14:00
382     2024-11-01 01:26:00
384     2024-11-01 01:24:00
383     2024-11-01 01:18:00
Name: news_first_upload_time, dtype: datetime64[ns]

In [14]:
import pyupbit


In [15]:
import pyupbit
import pandas as pd

# 데이터 가져오기
df3 = pyupbit.get_ohlcv("KRW-BTC", interval="minute60", count=1000,  to="2024-12-06 09:00:00")

# 11월 1일부터 최근까지 필터링
df3_filtered = df3.loc["2024-11-01":]

# 결과 확인
print(df3_filtered.head())
print(len(df3_filtered))

                           open        high         low       close  \
2024-11-01 00:00:00  98600000.0  99271000.0  98448000.0  98672000.0   
2024-11-01 01:00:00  98721000.0  99045000.0  98295000.0  98383000.0   
2024-11-01 02:00:00  98383000.0  99000000.0  98323000.0  98952000.0   
2024-11-01 03:00:00  98952000.0  98977000.0  98231000.0  98687000.0   
2024-11-01 04:00:00  98687000.0  98827000.0  98000000.0  98010000.0   

                         volume         value  
2024-11-01 00:00:00  314.510476  3.107732e+10  
2024-11-01 01:00:00  170.762422  1.682897e+10  
2024-11-01 02:00:00   93.157974  9.194855e+09  
2024-11-01 03:00:00   91.161494  8.982208e+09  
2024-11-01 04:00:00  113.523938  1.116089e+10  
858


In [16]:
df3_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 858 entries, 2024-11-01 00:00:00 to 2024-12-06 17:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    858 non-null    float64
 1   high    858 non-null    float64
 2   low     858 non-null    float64
 3   close   858 non-null    float64
 4   volume  858 non-null    float64
 5   value   858 non-null    float64
dtypes: float64(6)
memory usage: 46.9 KB


In [17]:
# 신문 523, 코인 858개 
# filtered_df, df3_filtered 이용하기

# api 키 가져오기

In [18]:
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API Key loaded successfully!")
else:
    print("API Key not found!")

API Key loaded successfully!


In [19]:
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

In [20]:
def get_completion(prompt, model = 'gpt-4o'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [21]:
def get_completion_4o_mini(prompt, model = 'gpt-4o-mini'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [22]:
def get_completion_turbo(prompt, model = 'gpt-3.5-turbo'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [23]:
def get_completion_3(prompt, model = "text-davinci-003"):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [24]:
def get_completion_ada(prompt, model = 'text-ada-001'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [25]:
def get_completion_codex(prompt, model = 'code-cushman-001'):
    messages = [{'role' : 'user', 'content' : prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

In [26]:
len(filtered_df[filtered_df['news_content'].str.contains('계엄')])


17

In [42]:
prompt = f"""
너는 신문기사를 분석하는 로봇이다. 내가 신문기사 523개가 담긴 데이터 프레임을 입력할 것이다. 
데이터 프레임을 바탕으로 나의 질문에 최대한 간략하게 답하여라.



<데이터 프레임> : <{filtered_df[filtered_df['news_content'].str.contains('계엄')]}>

<질문>
계엄령 사건에 대해서 알려줘. news_title을 보고 '계엄'이라는 단어가 보이면 news_content에 해당하는
내용들을 요약한 후 알려줘. 또한 계엄령 사건이 언제 일어났는지 알려줘.  


"""
print('4o')
response = get_completion(prompt)
print(response)

print('4o-mini')
response = get_completion_4o_mini(prompt)
print(response)

print('get_completion_turbo')
response = get_completion_turbo(prompt)
print(response)


4o
계엄령 사건은 윤석열 대통령이 2024년 12월 3일 오후에 비상 계엄령을 선포하면서 발생했습니다. 이로 인해 국내 주요 가상자산 거래소인 업비트와 빗썸이 마비되었고, 비트코인 등 가상자산 가격이 급락했습니다. 비트코인은 17% 가까이 급락했으며, 거래소 접속이 지연되는 등 혼란이 있었습니다. 그러나 이후 가상자산 시장은 안정세를 되찾았고, 비트코인 가격도 회복되었습니다. 일부 외국 투자자들은 이 상황을 저가 매수 기회로 활용하기도 했습니다.
4o-mini
계엄령 사건은 윤석열 대통령이 2024년 12월 3일에 비상 계엄령을 선포한 사건입니다. 이로 인해 국내 주요 가상자산 거래소인 업비트와 빗썸이 마비되었고, 비트코인 가격이 급락했습니다. 이후 가상자산 시장은 혼란을 겪었으나, 계엄령 해제 후 가격이 회복되는 모습을 보였습니다. 외국 투자자들도 이 상황을 노리고 가상자산을 매입하는 경향이 있었습니다. 

주요 내용 요약:
- 계엄령 선포 후 가상자산 가격 급락
- 업비트와 빗썸 거래소 마비
- 비트코인 가격이 17% 급락
- 계엄 해제 후 가격 회복 및 시장 안정화

계엄령 사건 발생일: 2024년 12월 3일.
get_completion_turbo
<답변>
계엄령 사건은 2024년 12월 3일에 발생했습니다. '계엄'이라는 단어가 포함된 기사들을 요약하면, 윤석열 대통령의 비상계엄 선포로 인해 국내 주요 가상자산 거래소들이 마비되거나 가격이 급락하는 현상이 발생했으며, 이에 따라 가상자산 시장이 혼란을 겪었지만 일부 코인들은 폭등하는 모습을 보였습니다. 계엄령 사태에도 글로벌 시세는 비교적 안정하게 유지되었고, 국내외 가상자산 시장은 혼란 없이 안정적으로 운영되었습니다.


In [28]:
prompt = f"""
너는 코인데이터를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 



<코인 데이터> : <{df3_filtered}>

<질문>
전체 자료 중, 최고가와 최저가에 대해 알려줘


"""
print('4o')
response = get_completion(prompt)
print(response)
print('--'*50)

print('4o-mini')
response = get_completion_4o_mini(prompt)
print(response)
print('--'*50)


print('get_completion_turbo')
response = get_completion_turbo(prompt)
print(response)

4o
주어진 코인 데이터에서 최고가와 최저가를 찾기 위해서는 'high' 열과 'low' 열을 각각 확인해야 합니다.

- 최고가 (high): 데이터에서 'high' 열의 최댓값을 찾으면 됩니다.
- 최저가 (low): 데이터에서 'low' 열의 최솟값을 찾으면 됩니다.

주어진 데이터에서:

- 최고가 (high): 138,637,000.0
- 최저가 (low): 98,000,000.0

이 값들은 각각의 열에서 가장 높은 값과 가장 낮은 값을 나타냅니다.
----------------------------------------------------------------------------------------------------
4o-mini
주어진 데이터에서 최고가와 최저가는 다음과 같습니다:

- **최고가 (High)**: 138637000.0
- **최저가 (Low)**: 98000000.0

이 값들은 데이터의 전체 기간 동안 기록된 최고가와 최저가입니다.
----------------------------------------------------------------------------------------------------
get_completion_turbo
전체 자료 중에서 최고가는 138637000이고, 최저가는 98000000입니다.


In [38]:

# 보고서 형태의 챗봇

prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 공신력있는 보고서 형태로 작성해줘
다만 <형식>에서 [[~~]]으로 되어 있는 부분은 너가 판단해서 넣어주라는 표시야.

<신문 기사> : <{filtered_df[filtered_df['news_content'].str.contains('계엄')]}>
<코인 데이터> : <{df3_filtered}>

<형식>
# 2024년 보고서: 계엄령이 가상자산 시장에 미친 영향 분석
# 발행일 : 2024년 n월 m일 00시 00분

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
   <<사건의 시작과 초기 시장 반응에 대해서 작성해줘>>
1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

   <<사건의 결과와 시장의 움직임은 어떻게 되었는지 알려줘.>>
2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

   <<사건의 결과로 어떤 현상/결과가 발생했는지 알려줘>>
3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   # 가격 변화 : <<'<코인 데이터>'에서 관련 기간을 가져와서 표로 보여줘 >>

-<<위에 있는 '가격 변화' 그래프를 기반으로 데이터 분석한 내용을 작성>>   

2. **거래량과 가치**:
   - 계엄령 발표 직후 거래소의 마비로 인해 거래량이 일시적으로 감소했을 가능성이 있습니다. 그러나 이후 저가 매수 기회로 인식되면서 거래량이 증가했을 것으로 보입니다.

## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급락은 외국 투자자들에게 저가 매수 기회를 제공하였고, 이는 역 김치프리미엄 현상을 초래했습니다. 전반적으로, 계엄령은 가상자산 시장의 변동성을 일시적으로 증가시켰으나, 시장의 회복력과 안정성을 확인할 수 있는 계기가 되었습니다.

## 정보 출처
- 신문 기사: Investing, Hankyung
- 코인 데이터: 2023년 1월 1일부터 12월 31일까지의 비트코인 가격 및 거래량 데이터

본 보고서는 계엄령이 가상자산 시장에 미친 영향을 종합적으로 분석하였으며, 향후 유사한 정치적 사건이 발생할 경우 시장의 반응을 예측하는 데 유용한 자료로 활용될 수 있습니다.






<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 '계엄'활동이 코인 가격에 어떤 충격을 줬는지 알려줘.


"""
print('4o')
response = get_completion(prompt)
print(response)
print('--'*50)

print('4o-mini')
response = get_completion_4o_mini(prompt)
print(response)
print('--'*50)


print('get_completion_turbo')
response = get_completion_turbo(prompt)
print(response)

4o
# 2024년 보고서: 계엄령이 가상자산 시장에 미친 영향 분석
# 발행일 : 2024년 [[12월 7일]] 00시 00분

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석

1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석

1. **가격 변동성**:
   - 가격 변화: 아래 표는 2024년 12월 3일부터 12월 6일까지의 비트코인 가격 변동을 보여줍니다.

   | 날짜 및 시간          | 시가        | 고가        | 저가        | 종가        |
   |-------------------|-----------|-----------|-----------|-----------|
   | 2024-12-03 22:00  | 1억 1,000만 원 | 1억 1,200만 원 | 9,400만 원  | 9,400만 원  |
   | 2024-12

In [39]:
# 주간 보고서 형태의 챗봇

prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 공신력있는 보고서 형태로 작성해줘
다만 <형식>에서 [[~~]]으로 되어 있는 부분은 너가 판단해서 넣어주라는 표시야.

<신문 기사> : <{filtered_df}>
<코인 데이터> : <{df3_filtered}>



<형식>
# 2024년 n월 m주 보고서: [[[금주에 가장 화제인 사건을 넣어줘]] 분석]

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 

# 현재가격 : 
# 가격 추세 : [[상승세 혹은 하락세]]
# 가격 변화 : [[코인 데이터를 기반으로 2024년 n월 m주에 있는 데이터를 일별로 정리해서 표로 보여줘]]
# 비트코인 시가총액 : 
# 거래종목 : 
# 거래소 : 

# 일별 가격 동향 : 
# 주간 가격 동향 비교 : 


## 신문 기사 분석
   ** 1주간 호재 **


   ** 1주간 악재 **

   
   ** 예상되는 전망 **

   
   ** 전문가의 중요 코멘트 ** 

   ** [[BUY OR SELL]]


## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급락은 외국 투자자들에게 저가 매수 기회를 제공하였고, 이는 역 김치프리미엄 현상을 초래했습니다. 전반적으로, 계엄령은 가상자산 시장의 변동성을 일시적으로 증가시켰으나, 시장의 회복력과 안정성을 확인할 수 있는 계기가 되었습니다.

## 정보 출처
- 신문 기사: Investing, Hankyung
- 코인 데이터: 2024년 1월 1일부터 12월 31일까지의 비트코인 가격 및 거래량 데이터

본 보고서는 계엄령이 가상자산 시장에 미친 영향을 종합적으로 분석하였으며, 향후 유사한 정치적 사건이 발생할 경우 시장의 반응을 예측하는 데 유용한 자료로 활용될 수 있습니다.










<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 12월의 첫째주 보고서를 작성해줘. 코인 데이터는 12월 1째주 자료를 가지고 작성.


"""
print('4o')
response = get_completion(prompt)
print(response)
print('--'*50)

print('4o-mini')
response = get_completion_4o_mini(prompt)
print(response)
print('--'*50)


print('get_completion_turbo')
response = get_completion_turbo(prompt)
print(response)

4o
# 2024년 12월 1주 보고서: 비트코인 10만 달러 재탈환 분석

## 서론
2024년 12월 첫째 주, 비트코인이 10만 달러를 재탈환하며 가상자산 시장에 큰 주목을 받았습니다. 이 보고서는 해당 주의 시장 동향과 주요 사건을 분석합니다.

# 현재가격 : 10만 달러
# 가격 추세 : 상승세
# 가격 변화 : 

| 날짜       | 시가       | 고가       | 저가       | 종가       | 거래량     |
|------------|------------|------------|------------|------------|------------|
| 2024-12-01 | 137,507,000 | 137,772,000 | 137,072,000 | 137,699,000 | 195.53 BTC |
| 2024-12-02 | 137,699,000 | 137,882,000 | 137,123,000 | 137,551,000 | 214.19 BTC |
| 2024-12-03 | 137,551,000 | 137,890,000 | 137,400,000 | 137,631,000 | 147.04 BTC |
| 2024-12-04 | 137,631,000 | 138,003,000 | 137,480,000 | 137,828,000 | 103.56 BTC |
| 2024-12-05 | 137,828,000 | 138,637,000 | 137,777,000 | 138,200,000 | 155.81 BTC |

# 비트코인 시가총액 : [[시가총액 계산 필요]]
# 거래종목 : 비트코인, 이더리움 등 주요 가상자산
# 거래소 : 빗썸, 업비트, 바이낸스 등

# 일별 가격 동향 : 비트코인은 주간 내내 상승세를 보이며 10만 달러를 재탈환했습니다.
# 주간 가격 동향 비교 : 전주 대비 상승세를 유지하며, 투자자들의 관심이 집중되었습니다.

## 신문 기사 분석
   ** 1주간 호재 **
   - 비트코인 10만 달러 재탈환: 시장의 긍정적 분위

In [40]:
df_trump = filtered_df[filtered_df['news_content'].str.contains('트럼프')]

df_trump.info()



<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 1 to 382
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   news_title              153 non-null    string        
 1   news_first_upload_time  153 non-null    datetime64[ns]
 2   news_content            153 non-null    string        
 3   news_url                153 non-null    string        
 4   news_website            153 non-null    string        
dtypes: datetime64[ns](1), string(4)
memory usage: 7.2 KB


In [41]:
# 보고서 형태의 챗봇

prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 공신력있는 보고서 형태로 작성해줘
다만 <형식>에서 [[~~]]으로 되어 있는 부분은 너가 판단해서 넣어주라는 표시야.

<신문 기사> : <{df_trump }>
<코인 데이터> : <{df3_filtered}>

<형식>
# 2024년 보고서: 계엄령이 가상자산 시장에 미친 영향 분석
# 발행일 : {formatted_time}

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
   <<사건의 시작과 초기 시장 반응에 대해서 작성해줘>>
1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

   <<사건의 결과와 시장의 움직임은 어떻게 되었는지 알려줘.>>
2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

   <<사건의 결과로 어떤 현상/결과가 발생했는지 알려줘>>
3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   # 가격 변화 : <<'<코인 데이터>'에서 관련 기간을 가져와서 표로 보여줘 >>

-<<위에 있는 '가격 변화' 그래프를 기반으로 데이터 분석한 내용을 작성>>   

2. **거래량과 가치**:
   - 계엄령 발표 직후 거래소의 마비로 인해 거래량이 일시적으로 감소했을 가능성이 있습니다. 그러나 이후 저가 매수 기회로 인식되면서 거래량이 증가했을 것으로 보입니다.

## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급락은 외국 투자자들에게 저가 매수 기회를 제공하였고, 이는 역 김치프리미엄 현상을 초래했습니다. 전반적으로, 계엄령은 가상자산 시장의 변동성을 일시적으로 증가시켰으나, 시장의 회복력과 안정성을 확인할 수 있는 계기가 되었습니다.

## 정보 출처
- 신문 기사: Investing, Hankyung
- 코인 데이터: 2023년 1월 1일부터 12월 31일까지의 비트코인 가격 및 거래량 데이터

본 보고서는 계엄령이 가상자산 시장에 미친 영향을 종합적으로 분석하였으며, 향후 유사한 정치적 사건이 발생할 경우 시장의 반응을 예측하는 데 유용한 자료로 활용될 수 있습니다.






<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 '트럼프의 대통령 선거'활동이 코인 가격에 어떤 충격을 줬는지 알려줘.


"""
print('4o')
response = get_completion(prompt)
print(response)
print('--'*50)

print('4o-mini')
response = get_completion_4o_mini(prompt)
print(response)
print('--'*50)


print('get_completion_turbo')
response = get_completion_turbo(prompt)
print(response)

4o
# 2024년 보고서: 트럼프 대통령 선거 활동이 가상자산 시장에 미친 영향 분석
# 발행일 : 2024-12-10-11:04:08

## 서론
2024년 미국 대통령 선거에서 도널드 트럼프의 활동은 글로벌 가상자산 시장에 상당한 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 트럼프의 선거 활동이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
1. **트럼프 선거 활동과 초기 시장 반응**:
   - 2024년 11월 1일, 트럼프의 선거 활동이 본격화되면서 비트코인 가격은 일시적으로 하락했습니다. 이는 트럼프의 정책 불확실성에 대한 투자자들의 우려로 해석됩니다.
   - "트럼프 승리 베팅 60%는 과장된 것, 실제는 박빙으로 예상"이라는 기사가 보도되면서 시장의 불확실성이 증가했습니다.

2. **시장 회복과 안정화**:
   - 이후 비트코인 가격은 점차 회복세를 보였으며, 12월 초에는 10만 달러를 재돌파하는 등 안정화되었습니다. 이는 트럼프의 정책 방향에 대한 명확성이 증가하면서 투자자들의 신뢰가 회복된 결과로 보입니다.

3. **정치적 불확실성과 시장 변동성**:
   - 트럼프의 선거 활동은 가상자산 시장의 변동성을 일시적으로 증가시켰습니다. 이는 정치적 불확실성이 투자자들의 심리에 영향을 미쳤기 때문입니다.

## 코인 데이터 분석
1. **가격 변동성**:
   - 2024년 11월 1일부터 12월 6일까지의 비트코인 가격 데이터를 분석한 결과, 트럼프의 선거 활동 기간 동안 가격 변동성이 증가한 것으로 나타났습니다. 
   - 예를 들어, 11월 1일 비트코인 가격은 98,672,000원에서 시작하여 12월 6일에는 138,200,000원으로 상승했습니다.

2. **거래량과 가치**:
   - 트럼프의 선거 활동 기간 동안 거래량은 일시적으로 감소했으나, 이후 회복세를 보였습니다. 이는 초기의 불확실성으로 인한 거래 감소와 이후의 시장 회복을 반영합니다.

## 결론
트럼프의 

In [43]:
# 속도 비교

In [45]:
import time

# 테스트 프롬프트
prompt = f"""
너는 코인데이터와 신문기사를 바탕으로 물음에 정확하게 답변하는 애널리스트이다. 공신력있는 보고서 형태로 작성해줘
다만 <형식>에서 [[~~]]으로 되어 있는 부분은 너가 판단해서 넣어주라는 표시야.

<신문 기사> : <{df_trump }>
<코인 데이터> : <{df3_filtered}>

<형식>
# 2024년 보고서: 계엄령이 가상자산 시장에 미친 영향 분석
# 발행일 : {formatted_time}

## 서론
2024년 12월 3일, 윤석열 대통령의 비상계엄령 선포는 국내외 가상자산 시장에 즉각적인 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 계엄령이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
   <<사건의 시작과 초기 시장 반응에 대해서 작성해줘>>
1. **계엄령 발표와 초기 반응**:
   - 2024년 12월 3일, 계엄령 발표 직후 비트코인 가격은 17% 급락했습니다. 이는 국내 주요 가상자산 거래소인 업비트와 빗썸의 접속 마비와 함께 발생했습니다.
   - 외국 투자자들은 이 급락을 저가 매수 기회로 인식하고, 스테이블코인 2300억 원이 업비트로 유입되었습니다.

   <<사건의 결과와 시장의 움직임은 어떻게 되었는지 알려줘.>>
2. **시장 회복과 안정화**:
   - 12월 4일, 비트코인 가격은 1억 3000만 원대로 회복되었으며, 계엄령 해제 후에는 1억 3500만 원대까지 상승했습니다.
   - 국내외 가상자산 시장은 계엄령의 혼란 여파 없이 빠르게 안정화되었습니다.

   <<사건의 결과로 어떤 현상/결과가 발생했는지 알려줘>>
3. **역 김치프리미엄 현상**:
   - 계엄령 발표 후 국내 비트코인 시세는 급락하며 '역 김치프리미엄'이 발생했습니다. 이는 해외 시장에 비해 국내 시장의 가격이 더 낮아지는 현상으로, 해외 자금의 유입을 촉진했습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   # 가격 변화 : <<'<코인 데이터>'에서 관련 기간을 가져와서 표로 보여줘 >>

-<<위에 있는 '가격 변화' 그래프를 기반으로 데이터 분석한 내용을 작성>>   

2. **거래량과 가치**:
   - 계엄령 발표 직후 거래소의 마비로 인해 거래량이 일시적으로 감소했을 가능성이 있습니다. 그러나 이후 저가 매수 기회로 인식되면서 거래량이 증가했을 것으로 보입니다.

## 결론
계엄령 발표는 국내 가상자산 시장에 단기적인 충격을 주었으나, 시장은 빠르게 회복되었습니다. 초기의 급락은 외국 투자자들에게 저가 매수 기회를 제공하였고, 이는 역 김치프리미엄 현상을 초래했습니다. 전반적으로, 계엄령은 가상자산 시장의 변동성을 일시적으로 증가시켰으나, 시장의 회복력과 안정성을 확인할 수 있는 계기가 되었습니다.

## 정보 출처
- 신문 기사: Investing, Hankyung
- 코인 데이터: 2023년 1월 1일부터 12월 31일까지의 비트코인 가격 및 거래량 데이터

본 보고서는 계엄령이 가상자산 시장에 미친 영향을 종합적으로 분석하였으며, 향후 유사한 정치적 사건이 발생할 경우 시장의 반응을 예측하는 데 유용한 자료로 활용될 수 있습니다.






<질문>
<신문 기사>, <코인 데이터> 자료를 바탕으로 '트럼프의 대통령 선거'활동이 코인 가격에 어떤 충격을 줬는지 알려줘.


"""

# GPT-4o 응답 시간 측정
print('4o')
start_time_4o = time.time()


response_4o = get_completion(prompt)
end_time_4o = time.time()
time_4o = end_time_4o - start_time_4o
print(response_4o)
print(f"GPT-4o 응답 시간: {time_4o:.2f}초")
print('--' * 50)

# GPT-4o-mini 응답 시간 측정
print('4o-mini')
start_time_4o_mini = time.time()
response_4o_mini = get_completion_4o_mini(prompt)
end_time_4o_mini = time.time()
time_4o_mini = end_time_4o_mini - start_time_4o_mini
print(response_4o_mini)
print(f"GPT-4o-mini 응답 시간: {time_4o_mini:.2f}초")
print('--' * 50)

# 응답 시간 비교
print(f"GPT-4o가 GPT-4o-mini보다 {time_4o - time_4o_mini:.2f}초 더 걸렸습니다.")


4o
# 2024년 보고서: 트럼프 대통령 선거 활동이 가상자산 시장에 미친 영향 분석
# 발행일 : 2024-12-10-11:04:08

## 서론
2024년 미국 대통령 선거에서 도널드 트럼프의 활동은 글로벌 가상자산 시장에 주목할 만한 영향을 미쳤습니다. 본 보고서는 신문 기사와 코인 데이터를 바탕으로 트럼프의 선거 활동이 가상자산, 특히 비트코인 가격에 미친 충격을 분석합니다.

## 신문 기사 분석
1. **트럼프 선거 활동과 초기 시장 반응**:
   - 2024년 11월 1일, 트럼프의 선거 활동이 본격화되면서 비트코인 가격은 일시적으로 하락했습니다. 이는 시장의 불확실성이 증가하면서 투자자들이 리스크 회피 심리를 보였기 때문입니다.
   - "트럼프 승리 베팅 60%는 과장된 것, 실제는 박빙으로 예상"이라는 기사가 보도되면서 시장의 불확실성이 더욱 커졌습니다.

2. **시장 회복과 안정화**:
   - 이후 비트코인 가격은 점차 회복세를 보였으며, 12월 초에는 10만 달러를 재돌파하는 등 강한 상승세를 나타냈습니다. 이는 트럼프의 선거 결과에 대한 불확실성이 해소되면서 시장이 안정화된 것으로 보입니다.

3. **시장 반응의 특징**:
   - 트럼프의 선거 활동은 초기에는 시장에 부정적인 영향을 미쳤으나, 이후 투자자들이 이를 기회로 인식하면서 시장이 빠르게 회복되었습니다.

## 코인 데이터 분석
1. **가격 변동성**:
   - 가격 변화: 2024년 11월 1일부터 12월 6일까지의 비트코인 가격 변동을 분석한 결과, 트럼프의 선거 활동이 시작된 11월 초에는 가격이 하락했으나, 이후 회복세를 보였습니다.

   | 날짜                | 시가        | 고가        | 저가        | 종가        |
   |-------------------|------------|------------|------------|------------|
   | 2024-11-01 00:00:00 | 98,600,